In [ ]:
#!git clone https://github.com/zhouhaoyi/Informer2020.git
#!git clone https://github.com/zhouhaoyi/ETDataset.git
# modify the code based on my dataset

In [ ]:
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install scikit_learn
# !{sys.executable} -m pip install torch
# !{sys.executable} -m pip install seaborn

In [1]:
import sys
import os
import pandas as pd
import numpy as np
from argparse import Namespace
import re
from sklearn.metrics import (
    mean_absolute_error,
    root_mean_squared_error,
    mean_absolute_percentage_error
)

In [2]:
if not 'Informer2020' in sys.path:
    sys.path += ['Informer2020']

from data.data_loader import Dataset_Custom
from utils.metrics import metric

In [3]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

2.8.0+cu128
12.8
True


In [4]:
num_gpus = torch.cuda.device_count()
print("Number of GPUs available:", num_gpus)

for i in range(num_gpus):
    gpu_name = torch.cuda.get_device_name(i)
    print(f"GPU {i}: {gpu_name}")

Number of GPUs available: 1
GPU 0: NVIDIA A30


## 1. Prepare the data

In [5]:
my_data = pd.read_csv('../GD030A_S.csv')

In [6]:
# Define the recover_timestamp function
def recover_timestamp(data):
    # Combine 'date' and 'time' to form a datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

    # Set 'datetime' as index
    data = data.set_index('datetime')

    # Create a complete range of timestamps with hourly frequency
    full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')

    # Reindex the data to include all timestamps, filling missing rows with NaN
    data_full = data.reindex(full_time_range)

    return data_full

Custom data (xxx.csv) has to include at least 2 features: date(format: YYYY-MM-DD hh:mm:ss) and target feature.

In [ ]:
traffic_full = recover_timestamp(my_data)
traffic_full.drop(['date', 'time'], axis=1, inplace=True)
traffic_full = traffic_full.reset_index(names='date')
traffic_full['date'] = traffic_full['date'].astype(str)
traffic_full

In [ ]:
#traffic_full.to_csv('traffic_full.csv', index=False)

In [60]:
# after covid data
traffic_full = pd.read_csv('../traffic_full.csv')
traffic_full
traffic_after_covid = traffic_full.loc[traffic_full[traffic_full['date'] == '2022-06-03 00:00:00'].index[0]:]
traffic_after_covid = traffic_after_covid.reset_index(drop=True)

In [59]:
#traffic_after_covid.to_csv('../traffic_after_covid.csv', index=False)

## 2. Run training directly using bash

In [5]:
# ensure the shell commands below using the current Python environment
os.environ["PATH"] = f"{sys.prefix}/bin:" + os.environ["PATH"]

In [6]:
cd Informer2020

/home/yl2672496l/Yue/code/transformer/Informer2020


In [7]:
#test
# !python -u main_informer.py --model informer --data traffic_after_covid --root_path ../ --features S --seq_len 24 --label_len 24 --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 1 #--inverse

In [8]:
seq_lens = [24 * i for i in range(1, 22)]
print(seq_lens)

[24, 48, 72, 96, 120, 144, 168, 192, 216, 240, 264, 288, 312, 336, 360, 384, 408, 432, 456, 480, 504]


### Train and test the model with different label_len
results1 with lable_len == 24, results with label_len == max(pred_len, seq_len // 3)

#### Entire dataset

In [ ]:
# for seq_len in seq_lens:
#     print(f"Running seq_len={seq_len}")
#     !python -u main_informer.py --model informer --data traffic_full --root_path ../ --features S --seq_len {seq_len} --label_len 24 --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 10 --train_epochs 20

for seq_len in seq_lens:
    # Calculate label_len: max(pred_len, seq_len // 3)
    label_len = max(6, seq_len // 3)
    
    print(f"Running seq_len={seq_len}, label_len={label_len}, pred_len=6")
    !python -u main_informer.py --model informer --data traffic_full --root_path ../ --features S --seq_len {seq_len} --label_len {label_len} --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 10 --train_epochs 20

Running seq_len=24, label_len=8, pred_len=6
Args in experiment:
Namespace(model='informer', data='traffic_full', root_path='../', data_path='traffic_full.csv', features='S', target='flow', freq='h', checkpoints='./checkpoints/', seq_len=24, label_len=8, pred_len=6, enc_in=1, dec_in=1, c_out=1, d_model=512, n_heads=8, e_layers=2, d_layers=1, s_layers=[3, 2, 1], d_ff=2048, factor=5, padding=0, distil=True, dropout=0.05, attn='prob', embed='timeF', activation='gelu', output_attention=False, do_predict=False, mix=True, cols=None, num_workers=0, itr=10, train_epochs=20, batch_size=32, patience=3, learning_rate=0.0001, des='Exp', loss='mse', lradj='type1', use_amp=False, inverse=False, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', detail_freq='h')
Use GPU: cuda:0
>>>>>>>start training : informer_traffic_full_ftS_sl24_ll8_pl6_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 21009
[DEBUG] Original dataset length: 21009
[DEBUG] Clean

#### After covid dataset

In [ ]:
# for seq_len in seq_lens:
#     print(f"Running seq_len={seq_len}")
#     !python -u main_informer.py --model informer --data traffic_after_covid --root_path ../ --features S --seq_len {seq_len} --label_len 24 --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 10 --train_epochs 20

for seq_len in seq_lens:
    # Calculate label_len: max(pred_len, seq_len // 3)
    label_len = max(6, seq_len // 3)
    
    print(f"Running seq_len={seq_len}, label_len={label_len}, pred_len=6")
    !python -u main_informer.py --model informer --data traffic_after_covid --root_path ../ --features S --seq_len {seq_len} --label_len {label_len} --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 10 --train_epochs 20

In [34]:
data = np.load("./results/running_time_traffic_full_24.npy")
data

array([13.86369028,  1.45002213])

## 3. Calculate the metrics

In [12]:
# Extract seq_len and run number from directory name
# Example:
# informer_traffic_full_ftS_sl24_ll24_pl6_dm512_..._Exp_3
# seq_len = 24, run_num = 3
def extract_info(dirname):
    seq_match = re.search(r"sl(\d+)", dirname)
    run_match = re.search(r"Exp_(\d+)", dirname)

    seq_len = int(seq_match.group(1)) if seq_match else None
    run_num = int(run_match.group(1)) + 1 if run_match else None  # Exp_0 → run 1

    return seq_len, run_num

# Compute metrics for each of 6 prediction steps
# pred, true shape: (N, 6, 1)
def compute_step_metrics(pred, true):
    steps = pred.shape[1]
    rows = []

    for step in range(steps):
        p = pred[:, step, 0]
        g = true[:, step, 0]

        mae = mean_absolute_error(g, p)
        rmse = root_mean_squared_error(g, p)
        mape = mean_absolute_percentage_error(g, p) * 100  # convert to percentage
        rows.append((step + 1, mae, rmse, mape))
        
    return rows

#### Metrics of each run and each step (6×10×21)

In [13]:
def whole_metrics(target_dataset):
    all_rows = []
    
    # your result directories (you can replace this with auto-search)
    result_dirs = sorted([
        d for d in os.listdir("results")
        if os.path.isdir(os.path.join("results", d)) and f"informer_{target_dataset}_" in d  # <<< key filtering step
    ])
    
    for d in result_dirs:
        seq_len, run_num = extract_info(d)
    
        pred_path = os.path.join("results", d, "pred_inverse.npy")
        true_path = os.path.join("results", d, "true_inverse.npy")
    
        if not (os.path.exists(pred_path) and os.path.exists(true_path)):
            print(f"Skipping missing: {d}")
            continue
    
        pred = np.load(pred_path)
        true = np.load(true_path)
    
        metrics = compute_step_metrics(pred, true)
    
        for step, mae, rmse, mape in metrics:
            all_rows.append({
                "input_len": seq_len,
                "run_num": run_num,
                "pre_step": step,
                "MAE_test": mae,
                "RMSE_test": rmse,
                "MAPE (%)_test": mape
            })
    
    df = pd.DataFrame(all_rows)
    df = df.sort_values(by=["input_len", "run_num", "pre_step"]).reset_index(drop=True)

    return df

In [14]:
metrics_full_entire_data = whole_metrics("traffic_full")
metrics_full_entire_data
#metrics_full_entire_data.to_csv('../all_metrics_informer_entire_data.csv', index=False)

,input_len,run_num,pre_step,MAE_test,RMSE_test,MAPE (%)_test
0,24,1,1,22.700678,33.696602,25.014281
1,24,1,2,25.752703,38.767822,29.830268
2,24,1,3,26.951324,40.767159,32.020304
3,24,1,4,27.492552,41.845589,33.026516
4,24,1,5,28.320366,42.851768,35.030124
...,...,...,...,...,...,...
1255,504,10,2,28.688896,44.067169,38.000548
1256,504,10,3,30.025515,47.222652,39.781231
1257,504,10,4,31.670654,50.132671,42.744172
1258,504,10,5,31.289970,50.489079,40.815246


In [15]:
metrics_full_after_covid = whole_metrics("traffic_after_covid")
metrics_full_after_covid
#metrics_full_after_covid.to_csv('../all_metrics_informer_after_covid.csv', index=False)

,input_len,run_num,pre_step,MAE_test,RMSE_test,MAPE (%)_test
0,24,1,1,21.706732,28.366425,20.096470
1,24,1,2,25.141886,33.365131,22.878578
2,24,1,3,27.501200,36.124100,24.968128
3,24,1,4,28.268160,37.313774,25.094458
4,24,1,5,27.421959,36.910595,25.243387
...,...,...,...,...,...,...
1255,504,10,2,27.076494,34.429127,25.611803
1256,504,10,3,31.788719,39.325233,28.024593
1257,504,10,4,42.202965,52.031471,37.160045
1258,504,10,5,38.867172,48.012154,34.091556


#### Metrics of each input length (21)

In [32]:
#label_len == max(pred_len, seq_len // 3)
metrics_entire_data = metrics_full_entire_data.groupby("input_len")[["MAE_test", "RMSE_test", "MAPE (%)_test"]].mean().reset_index()
metrics_entire_data

,input_len,MAE_test,RMSE_test,MAPE (%)_test
0,24,26.944691,39.958597,33.937828
1,48,27.081740,40.409057,34.893205
2,72,27.446016,40.553551,36.966954
3,96,27.551050,41.393979,35.007789
4,120,27.421511,41.112958,36.939158
5,144,27.713128,41.471491,36.701259
6,168,27.322957,40.987236,36.373228
7,192,27.113147,41.241941,34.820253
8,216,27.703049,41.704202,37.449192
9,240,27.250186,42.097064,35.684882


In [20]:
#lable_len == 24
metrics_entire_data = metrics_full_entire_data.groupby("input_len")[["MAE_test", "RMSE_test", "MAPE (%)_test"]].mean().reset_index()
metrics_entire_data
#metrics_entire_data.to_csv('../metrics_informer_entire_data.csv', index=False)

,input_len,MAE_test,RMSE_test,MAPE (%)_test
0,24,26.941581,40.207250,33.782750
1,48,27.139905,40.404166,35.068444
2,72,27.189945,40.615940,34.702683
3,96,27.661340,41.291684,36.436086
4,120,27.578414,41.291757,36.374518
5,144,27.827786,41.772902,37.024470
6,168,27.727947,41.254727,35.691959
7,192,27.442648,41.235187,36.364209
8,216,27.334847,41.454571,35.961914
9,240,27.358569,41.704306,36.569286


In [18]:
#label_len == max(pred_len, seq_len // 3)
metrics_after_covid = metrics_full_after_covid.groupby("input_len")[["MAE_test", "RMSE_test", "MAPE (%)_test"]].mean().reset_index()
metrics_after_covid

,input_len,MAE_test,RMSE_test,MAPE (%)_test
0,24,26.612306,34.547977,24.540002
1,48,27.214656,35.467811,24.924143
2,72,30.125909,38.647433,27.914294
3,96,29.846185,38.436569,27.761817
4,120,29.525302,38.107861,29.072096
5,144,31.280961,39.798043,30.327030
6,168,32.076638,40.467207,32.460847
7,192,33.703165,42.489039,33.613593
8,216,28.080678,35.515493,30.987297
9,240,29.132885,36.759904,31.976971


In [21]:
#lable_len == 24
metrics_after_covid = metrics_full_after_covid.groupby("input_len")[["MAE_test", "RMSE_test", "MAPE (%)_test"]].mean().reset_index()
#metrics_after_covid.to_csv('../metrics_informer_after_covid.csv', index=False)
metrics_after_covid

,input_len,MAE_test,RMSE_test,MAPE (%)_test
0,24,26.612306,34.547977,24.540002
1,48,27.214656,35.467811,24.924143
2,72,30.125909,38.647433,27.914294
3,96,29.846185,38.436569,27.761817
4,120,29.525302,38.107861,29.072096
5,144,31.280961,39.798043,30.327030
6,168,32.076638,40.467207,32.460847
7,192,33.703165,42.489039,33.613593
8,216,28.080678,35.515493,30.987297
9,240,29.132885,36.759904,31.976971


### Compare the metrics with model built in results

In [22]:
# Collect all result directories
target_dataset = "traffic_after_covid"
result_dirs = sorted([
    d for d in os.listdir("results")
    if os.path.isdir(os.path.join("results", d)) and f"informer_{target_dataset}_" in d  # <<< key filtering step
])

# Dictionary to store metrics per seq_len
metrics_dict = {}

for d in result_dirs:
    folder_path = os.path.join("results", d)
    
    metrics_path = os.path.join(folder_path, "metrics_inverse.npy")
    if not os.path.exists(metrics_path):
        print(f"[WARNING] Missing metrics_inverse.npy in {folder_path}")
        continue
    
    metrics = np.load(metrics_path)  # [MAE, MSE, RMSE, MAPE, MSPE]
    
    # Extract seq_len from folder name (assuming folder name contains 'sl<seq_len>_run<run_num>')
    seq_len = int([s for s in d.split("_") if s.startswith("sl")][0][2:])
    
    if seq_len not in metrics_dict:
        metrics_dict[seq_len] = {"MAE": [], "RMSE": [], "MAPE": []}
    
    metrics_dict[seq_len]["MAE"].append(metrics[0])
    metrics_dict[seq_len]["RMSE"].append(metrics[2])
    metrics_dict[seq_len]["MAPE"].append(metrics[3])

# Calculate mean over all runs per seq_len
rows = []
for seq_len, vals in metrics_dict.items():
    rows.append({
        "input_len": seq_len,
        "MAE_test": np.mean(vals["MAE"]),
        "RMSE_test": np.mean(vals["RMSE"]),
        "MAPE (%)_test": np.mean(vals["MAPE"]) *100
    })

df_inverse_avg = pd.DataFrame(rows).sort_values("input_len").reset_index(drop=True)
df_inverse_avg

,input_len,MAE_test,RMSE_test,MAPE (%)_test
0,24,26.918604,34.910751,24.879768
1,48,27.556540,35.952579,24.921795
2,72,29.252844,37.781456,27.301199
3,96,30.217615,38.881721,28.015249
4,120,30.552286,39.505878,29.846334
5,144,32.854408,41.626030,31.584963
6,168,32.003563,40.360962,31.648952
7,192,33.417553,42.187721,34.222759
8,216,28.220301,35.503487,31.007893
9,240,28.169607,35.782070,31.431999


In [49]:
avg_by_seq.compare(df_inverse_avg)  

MAE_test             RMSE_test            MAPE (%)_test           
         self      other       self      other          self      other
0   26.827424  26.827423  40.091506  40.248596     33.976340  33.976345
1   27.086889  27.086889  40.371117  40.532936     35.322956  35.322960
2   27.484757  27.484756  40.414853  40.567635     37.142969  37.142967
3   27.565523  27.565521  41.038521  41.195618     35.855909  35.855907
4   27.364081  27.364080  41.213944  41.375389     35.366292  35.366291
5   27.528013  27.528015  41.292334  41.451664     35.943623  35.943626
6   27.900328  27.900330  41.459094  41.608902     37.013240  37.013237
7   27.311530  27.311533  41.252090  41.407749     34.981317  34.981316
8   27.348362  27.348362  40.972219  41.117546     37.868599  37.868603
9   27.662873  27.662872  41.741723  41.893860     37.931720  37.931721
10  27.909886  27.909887  42.613484  42.781487     36.033699  36.033703
11  28.428512  28.428513  42.839811  43.016197     38.538594  38.538597
12  27.657373  27.657373  42.875749  43.059753     34.077934  34.077934
13  28.042166  28.042166  42.898795  43.075539     35.931267  35.931267
14  28.378985  28.378986  43.788344  43.971962     34.976826  34.976826
15  28.860072  28.860071  44.196937  44.398430     36.085584  36.085587
16  29.124343  29.124344  44.124789  44.320198     38.010948  38.010944
17  28.937098  28.937099  44.651621  44.846455     36.279675  36.279671
18  29.117829  29.117828  44.748922  44.940174     37.068310  37.068310
19  29.624438  29.624439  46.003536  46.211472     36.347774  36.347771
20  29.368494  29.368494  46.124282  46.341087     36.350152  36.350151

## 4. Calculate the model size

In [ ]:
from models.model import Informer  # ← change import path if needed

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

if __name__ == "__main__":
    # ---- MODEL CONFIG ----
    model = Informer(
        enc_in=1,
        dec_in=1,
        c_out=1,
        seq_len=24,
        label_len=24,
        out_len=6,
        d_model=512,
        n_heads=8,
        e_layers=2,
        d_layers=1,
        d_ff=2048,
        dropout=0.05,
        attn='prob',
        embed='timeF',
        freq='h',
        distil=True,
        #device=torch.device("cpu")
    )

    # ---- COUNT ----
    total = count_parameters(model)
    print(f"Total trainable parameters: {total:,}")

## 5. Calculate the running time

In [36]:
def combine_npy_by_prefix(directory, prefix):
    """
    Scans the given directory for files matching:
        <prefix>_<seq_len>.npy
    Loads each file (expected to contain [training_time, inference_time]),
    and writes a CSV named <prefix>.csv
    """
    # regex: prefix_seqLen.npy
    # escaping prefix just in case it contains regex characters
    pattern = re.compile(r"(" + re.escape(prefix) + r")_(\d+)\.npy")

    records = []

    for fname in os.listdir(directory):
        match = pattern.match(fname)
        if not match:
            continue

        seq_len = int(match.group(2))
        arr = np.load(os.path.join(directory, fname))

        if len(arr) < 2:
            raise ValueError(f"File {fname} does not contain at least two values.")

        training_time, inference_time = arr[0], arr[1]
        records.append((seq_len, training_time, inference_time))

    if not records:
        raise ValueError("No matching files found for prefix: " + prefix)

    df = pd.DataFrame(records, columns=["seq_len", "training_time", "inference_time"])
    df = df.sort_values("seq_len")

    # out_csv = os.path.join(directory, prefix + ".csv")
    # df.to_csv(out_csv, index=False)
    return df

In [43]:
running_time_entire_data = combine_npy_by_prefix("./results", "running_time_traffic_full")
running_time_entire_data
# running_time_entire_data.to_csv('../running_time_informer_entire_data.csv', index=False)

In [44]:
running_time_after_covid = combine_npy_by_prefix("./results", "running_time_traffic_after_covid")
# running_time_after_covid.to_csv('../running_time_informer_after_covid.csv', index=False)